Gestión de las bases de datos

In [75]:
import pandas as pd
import numpy as np
import os

Definición del __init__ de la clase

In [59]:
class DatabaseManager:
    """Gestor de bases de datos de perfiles estructurales"""
    
    def __init__(self, data_folder=r'..\database'):
        self.data_folder = data_folder
        self.db_aisc = None
        self.db_cirsoc = None
        self.current_db = 'AISC'
        self.load_databases()
    
    def load_databases(self):
        """Cargar bases de datos desde CSV (excluyendo ángulos)"""
        
        # AISC
        try:
            aisc_path = os.path.join(self.data_folder, 'perfiles_SI.csv')
            self.db_aisc = pd.read_csv(aisc_path)
            
            
            # Limpiar nombres de columnas
            self.db_aisc.columns = self.db_aisc.columns.str.strip()
            
            print(f"✓ AISC cargada: {len(self.db_aisc)} perfiles (sin ángulos)")
            print(f"  Familias disponibles: {sorted(self.db_aisc['Tipo'].unique().tolist())}")
            
        except Exception as e:
            print(f"⚠️ Error al cargar AISC: {e}")
            self.db_aisc = pd.DataFrame()
        
        # Cargar CIRSOC
        try:
            cirsoc_path = os.path.join(self.data_folder, 'cirsoc-shapes-database.csv')
            self.db_cirsoc = pd.read_csv(cirsoc_path)
            
            # Limpiar nombres de columnas
            self.db_cirsoc.columns = self.db_cirsoc.columns.str.strip()
            
            print(f"✓ CIRSOC cargada: {len(self.db_cirsoc)} perfiles (sin ángulos)")
            print(f"  Familias disponibles: {sorted(self.db_cirsoc['Tipo'].unique().tolist())}")
            
        except Exception as e:
            print(f"⚠️ Error al cargar CIRSOC: {e}")
            self.db_cirsoc = pd.DataFrame()
    
    def set_database(self, db_name):
        """Cambiar base de datos activa"""
        db_name = db_name.upper()
        if db_name in ['AISC', 'CIRSOC']:
            self.current_db = db_name
            print(f"✓ Base de datos activa: {self.current_db}")
        else:
            raise ValueError("Base de datos debe ser 'AISC' o 'CIRSOC'")
    
    def get_current_database(self):
        """Obtener base de datos activa"""
        if self.current_db == 'AISC':
            return self.db_aisc.copy()
        else:
            return self.db_cirsoc.copy()
    
    def get_database_name(self):
        """Obtener nombre de la base de datos activa"""
        return self.current_db
    
    def get_familias(self):
        """Obtener lista de familias de perfiles """
        db = self.get_current_database()
        
        if 'Tipo' in db.columns:
            familias = sorted([f for f in db['Tipo'].unique()])
            return familias
        
        return []
    
    def get_perfiles_by_familia(self, familia):
        """Obtener perfiles de una familia específica"""
        
        db = self.get_current_database()
        
        if 'Tipo' in db.columns:
            perfiles = db[db['Tipo'] == familia]['PERFIL'].tolist()
            return sorted(perfiles)
        
        return []
    
    def get_perfil_data(self, perfil_nombre):
        """Obtener datos completos de un perfil"""
        db = self.get_current_database()
        
        try:
            perfil = db[db['PERFIL'] == perfil_nombre].iloc[0]
            
            return perfil
            
        except IndexError:
            raise ValueError(f"Perfil '{perfil_nombre}' no encontrado en base de datos {self.current_db}")
        except Exception as e:
            raise ValueError(f"Error al obtener perfil '{perfil_nombre}': {e}")
    
    def get_perfiles_aplicables_flexion(self):
        """Obtener perfiles aplicables para cálculo de flexión """
        db = self.get_current_database()
        
        # Solo perfiles con alas y sin ángulos
        if 'Tipo' in db.columns:
            # Tipos aplicables a flexión (excluyendo ángulos y tubos redondos)
            tipos_flexion = ['W', 'S', 'M', 'C', 'MC', 'HP','IPBv','IPB','UPN','IPBl','IPN','IPE']
            perfiles_flexion = db[db['Tipo'].isin(tipos_flexion)].copy()
            return perfiles_flexion
        
        return db
    
    def get_perfiles_aplicables_compresion(self):
        """Obtener todos los perfiles aplicables a compresión (sin ángulos)"""
        db = self.get_current_database()
        
        return db
    
    def get_info_perfil(self, perfil_nombre):
        """Obtener información resumida de un perfil para display"""
        try:
            perfil = self.get_perfil_data(perfil_nombre)
            
            info = {
                'nombre': perfil_nombre,
                'tipo': perfil.get('Tipo', 'N/A'),
                'peso': perfil.get('W', np.nan),
                'area': perfil.get('A', np.nan),
                'altura': perfil.get('d', np.nan),
                'ancho': perfil.get('bf', np.nan),
                'clasificacion_alas': perfil.get('Clasificacion_Alas', 'N/A'),
                'clasificacion_alma': perfil.get('Clasificacion_Alma', 'N/A')
            }
            
            return info
            
        except Exception as e:
            return None
    
    def buscar_perfiles(self, criterio, valor_min=None, valor_max=None):
        """
        Buscar perfiles según criterio
        
        Ejemplos:
        - buscar_perfiles('W', valor_min=50, valor_max=100)  # Peso entre 50-100 kg/m
        - buscar_perfiles('d', valor_min=300, valor_max=500) # Altura entre 300-500 mm
        """
        db = self.get_current_database()
        
        if criterio == 'Tipo':
            # Buscar por tipo
            if valor_min:
                return db[db['Tipo'] == valor_min].copy()
        elif criterio in db.columns:
            # Buscar por rango
            resultado = db.copy()
            if valor_min is not None:
                resultado = resultado[resultado[criterio] >= valor_min]
            if valor_max is not None:
                resultado = resultado[resultado[criterio] <= valor_max]
            return resultado
        
        return pd.DataFrame()
    
    def get_estadisticas(self):
        """Obtener estadísticas de la base de datos activa"""
        db = self.get_current_database()
        
        stats = {
            'total_perfiles': len(db),
            'familias': db['Tipo'].nunique() if 'Tipo' in db.columns else 0,
            'peso_min': db['W'].min() if 'W' in db.columns else np.nan,
            'peso_max': db['W'].max() if 'W' in db.columns else np.nan,
            'altura_min': db['d'].min() if 'd' in db.columns else np.nan,
            'altura_max': db['d'].max() if 'd' in db.columns else np.nan
        }
        
        return stats
    
    def exportar_seleccion(self, perfiles_nombres, filename='seleccion_perfiles.csv'):
        """Exportar datos de perfiles seleccionados a CSV"""
        db = self.get_current_database()
        seleccion = db[db['PERFIL'].isin(perfiles_nombres)].copy()
        seleccion.to_csv(filename, index=False)
        print(f"✓ {len(seleccion)} perfiles exportados a {filename}")
        return filename



In [60]:


# ============================================================================
# FUNCIONES DE CONVENIENCIA
# ============================================================================

def cambiar_base_datos(db_name):
    """Función de conveniencia para cambiar base de datos"""
    db_manager.set_database(db_name)

def obtener_familias():
    """Función de conveniencia para obtener familias"""
    return db_manager.get_familias()

def obtener_perfiles_familia(familia):
    """Función de conveniencia para obtener perfiles de una familia"""
    return db_manager.get_perfiles_by_familia(familia)

def obtener_datos_perfil(perfil_nombre):
    """Función de conveniencia para obtener datos de un perfil"""
    return db_manager.get_perfil_data(perfil_nombre)

In [61]:
# ============================================================================
# INSTANCIA GLOBAL
# ============================================================================

# Crear instancia global del gestor de bases de datos
db_manager = DatabaseManager()

✓ AISC cargada: 2091 perfiles (sin ángulos)
  Familias disponibles: ['2L', 'C', 'HP', 'HSS', 'L', 'M', 'MC', 'MT', 'PIPE', 'S', 'ST', 'W', 'WT']
✓ CIRSOC cargada: 514 perfiles (sin ángulos)
  Familias disponibles: ['C', 'HP', 'IPB', 'IPBl', 'IPBv', 'IPE', 'IPN', 'L', 'M', 'MC', 'UPN', 'W']


In [63]:
cambiar_base_datos('CIRSOC')

✓ Base de datos activa: CIRSOC


In [64]:
obtener_familias()

['C', 'HP', 'IPB', 'IPBl', 'IPBv', 'IPE', 'IPN', 'L', 'M', 'MC', 'UPN', 'W']

In [68]:
obtener_perfiles_familia('L')

[' L 1 3/4 x 1 3/4 x 3/16',
 'L 1 1/2 x 1 1/2 x 1/4',
 'L 1 1/2 x 1 1/2 x 1/8',
 'L 1 1/2 x 1 1/2 x 3/16',
 'L 1 1/4 x 1 1/4 x 1/4',
 'L 1 1/4 x 1 1/4 x 1/8',
 'L 1 1/4 x 1 1/4 x 3/16',
 'L 1 1/8 x 1 1/8 x 1/8',
 'L 1 3/4 x 1 3/4 x 1/4',
 'L 1 3/4 x 1 3/4 x 1/8',
 'L 1x 1 x 1/4',
 'L 1x 1 x 1/8',
 'L 1x 1 x 3/16',
 'L 2 1/2 x 2 1/2 x 1/4',
 'L 2 1/2 x 2 1/2 x 3/16',
 'L 2 1/2 x 2 1/2 x 3/8',
 'L 2 1/2 x 2 1/2 x 5/16',
 'L 2 1/4 x 2 1/4 x 1/4',
 'L 2 1/4 x 2 1/4 x 1/8',
 'L 2 1/4 x 2 1/4 x 3/16',
 'L 2 x 2 x 1/4',
 'L 2 x 2 x 1/8',
 'L 2 x 2 x 3/16',
 'L 2 x 2 x 3/8',
 'L 2 x 2 x 5/16',
 'L 3 1/2 x 3 1/2 x 1/2',
 'L 3 1/2 x 3 1/2 x 1/4',
 'L 3 1/2 x 3 1/2 x 3/8',
 'L 3 1/2 x 3 1/2 x 5/16',
 'L 3 x 3 x 1/2',
 'L 3 x 3 x 1/4',
 'L 3 x 3 x 3/8',
 'L 3 x 3 x 5/16',
 'L 3/4 x 3/4 x 1/8',
 'L 4 x 4 x 1/2',
 'L 4 x 4 x 1/4',
 'L 4 x 4 x 3/8',
 'L 4 x 4 x 5/16',
 'L 4 x 4 x 7/16',
 'L 5 x 5 x 1/2',
 'L 5 x 5 x 3/8',
 'L 5 x 5 x 7/16',
 'L 5/8 x 5/8 x 1/8',
 'L 6 x 6 x 1/2',
 'L 6 x 6 x 3/8',
 '

In [69]:
db_manager.get_perfil_data('L 2 x 2 x 1/4')

Tipo                         L
PERFIL           L 2 x 2 x 1/4
d                          NaN
bf                         NaN
tf                         NaN
hw                         NaN
tw-r1                      NaN
r2                         NaN
bf/2tf                     NaN
hw/tw                      NaN
Ag                        6.17
Peso                      4.84
Ix                         NaN
Sx                         NaN
rx                         NaN
Qx                         NaN
Zx                         NaN
Iy                         NaN
Sy                         NaN
ry                         NaN
Qy                         NaN
1.5Sy                      NaN
Zy                         NaN
w1                         NaN
d1                         NaN
w4                         NaN
t1                         NaN
t2                         NaN
J                         0.83
Cw                        1.57
X1                         NaN
X2 (10)-5                  NaN
Lp_alma 

df = pd.rea

In [70]:
df = pd.read_csv('../database/cirsoc-shapes-database.csv')

In [74]:
df.groupby('Tipo').count().to_csv('count_variables.csv')